In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import re
import os
import sklearn
import tqdm
from tqdm import tqdm
import nltk
from PIL import Image
import matplotlib.pyplot as plt
from skimage.transform import resize
import warnings
import skimage.io
import selective_search
warnings.filterwarnings("ignore") 

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

import cv2
import json

In [2]:
#### listing all jpeg files
def list_jpg_files(directory):
    jpg_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".jpeg"):
                jpg_files.append(os.path.join(root, file))
    return jpg_files

directory_path = "files"
jpg_files = list_jpg_files(directory_path)
print("JPEG files found:")
for jpg_file in jpg_files:
    print(jpg_file)

JPEG files found:
files/p11/p11000183/s50336039/3761aae0-255c0808-86d2121b-88ae172f-b7625d50.jpg
files/p11/p11000011/s51029426/ff213473-b64efa18-863f2bad-76181481-30bc30d7.jpg
files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg
files/p10/p10000032/s50414267/174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.jpg
files/p10/p10000764/s57375967/b79e55c3-735ce5ac-64412506-cdc9ea79-f1af521f.jpg
files/p10/p10000764/s57375967/096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4.jpg


In [3]:
split_train = pd.read_csv("mimic-cxr-2.0.0-split.csv")

In [7]:
split_train[split_train["study_id"] == 59816154]

,dicom_id,study_id,subject_id,split
351914,97b6cad9-5a22e474-39ab224b-4d22949b-8d5d5c96,59816154,19336651,train
351915,e95a839b-e46897ef-0a05e1cd-4674f0a9-e330eeb4,59816154,19336651,train


In [11]:
list_sub_train = split_train[split_train['split'] == 'train']['subject_id'].unique()

In [23]:
split_train[split_train['subject_id'].isin(list_train_subject_abn)]

,dicom_id,study_id,subject_id,split
10,0c4eb1e1-b801903c-bcebe8a4-3da9cd3c-3b94a27c,50771383,10000898,train
11,2a280266-c8bae121-54d75383-cac046f4-ca37aa16,50771383,10000898,train
12,8959e402-2175d68d-edba5a6c-baab51c3-9359f700,54205396,10000898,train
13,9e7a6aae-2580e589-6212d336-9813ebbd-a9239a34,54205396,10000898,train
14,b75df1bd-0f22d631-52d73526-2ae7b85a-d843b39d,54205396,10000898,train
...,...,...,...,...
377062,e3e5f8c3-14429887-b1181522-445ec3a8-110663c3,52252154,19998562,train
377063,6ef0dbd3-7de20893-5fbcafb8-8f32e0d7-30a1150b,56823087,19998562,train
377064,b70ca192-74b20b3e-669f166c-beec4177-2ece8df2,56823087,19998562,train
377065,12fad032-cf4eff00-32a4fc65-67e424b6-a2e42afb,59764731,19998562,train


In [14]:
file = open("cluster_annotation.json")
mimic_abn = json.load(file)

In [22]:
list_train_subject_abn = []
for i in mimic_abn["train"]:
    str_subject = str(i["id"]).split("_")
    str_subject = str_subject[0]
    list_train_subject_abn.append(int(str_subject))

In [4]:
### get all regions
directory = 'scene_graph'

list
length_data = 0
list_regions = ["right lung", "right costophrenic angle","right cardiophrenic angle","right hemidiaphragm",
                "left lung", "left costophrenic angle","left hemidiaphragm","trachea","right clavicle",
               "left clavicle","aortic arch","mediastinum","cavoatrial junction","left cardiophrenic angle",
               "right atrium"]
dict_image = {}
no_region_image_id = []
length_data_relation = 0
for filename in os.scandir(directory):
    name = filename.name.split("_")[0]
    if filename.path.endswith(".json"):
        for image_path in jpg_files:
            path_name = image_path.split("/")[-1].split(".")[0]
            if path_name == name :
                length_data = length_data +1
                f = open(filename)
                data = json.load(f)
                list_region = []
                count = len(data["objects"]
                for region in data["objects"]:
                    if region["bbox_name"] in list_regions:
                        list_region.append([region["original_x1"], region["original_y1"], region["original_x2"],region["original_y2"]])
                dict_image[image_path] = list_region
           
                

In [6]:
for i in dict_image:
    print(i)
    print(len(dict_image[i]))
    print("==================")
print(len(list_regions))

files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg
15
files/p10/p10000764/s57375967/096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4.jpg
15
files/p11/p11000011/s51029426/ff213473-b64efa18-863f2bad-76181481-30bc30d7.jpg
15
files/p11/p11000183/s50336039/3761aae0-255c0808-86d2121b-88ae172f-b7625d50.jpg
15
16


In [54]:
### cropping all images
def region_feature_preprocessing(image, bounding_box):

    cropped_image = image.crop(bounding_box)
    image_array = np.array(cropped_image)
    padded_image = image_array
    
    height, width = cropped_image.size 
    
    max_pad = max(height, width)
    if height < max_pad :
        pad_height = max_pad - height
        padded_image = np.pad(image_array, ((0, pad_height), (0,0),(0,0)), mode='constant')
        
    if width < max_pad :
        pad_width = max_pad - width
        padded_image = np.pad(image_array, ((0, 0), (0,pad_width), (0,0)), mode='constant')
    return Image.fromarray(padded_image)


#image_inputs = {}
#for image in dict_image:
#    aa = region_feature_preprocessing(image, dict_image[image])
#    image_inputs[image] = aa



In [17]:
len(image_inputs[0])

15

In [118]:
region_input_tensors = torch.tensor(image_inputs, dtype=torch.float32)

In [119]:
region_input_tensors.shape

torch.Size([4, 36, 1024])

In [12]:
### trying densenet121 on original image 224x224

model = models.densenet121(pretrained=True)

# Remove the original classifier (fully connected layers)
model_features = torch.nn.Sequential(*list(model.children())[:-1])

# Define global average pooling layer
global_avg_pooling = nn.AdaptiveAvgPool2d((1, 1))  # Perform global average pooling to (1, 1)


# Combine the layers in a Sequential module
model = nn.Sequential(model_features, global_avg_pooling, nn.Flatten())

# Set the model to evaluation mode
model.eval()

Sequential(
  (0): Sequential(
    (0): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine

In [ ]:
#### extracting the image features from the original image
def image_global_feature_extraction(image_path, model):
    transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
    ])

    # Load and preprocess the input image
    image = Image.open(image_path)
    image = image.convert("RGB")
    print(type(image))
    image_tensor = transform(image).unsqueeze(0)
    extracted_features = model(image_tensor)
    return extracted_features

def image_local_feature_extraction(image_path, bounding_boxes, model):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to 224x224
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
    ])
    region_features = []    
    # Load and preprocess the input image
    image = Image.open(image_path)
    image = image.convert("RGB")
    for region in bounding_boxes:
        cropped_image = region_feature_preprocessing(image, region)  # Crop the image
        image_tensor = transform(cropped_image).unsqueeze(0)  # Apply transformations
        extracted_features = model(image_tensor)
        region_features.append(extracted_features)
    concat_region = torch.cat(region_features, dim=0)    
    return concat_region



full_image_features = []
for image in dict_image:
    global_features = image_global_feature_extraction(image,model)
    local_features = image_local_feature_extraction(image,dict_image[image], model)
    combined_features = torch.cat((global_features, local_features), dim=0)
    full_image_features.append(combined_features)

input_features = torch.stack(full_image_features, dim=0)
input_features.shape


<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>


In [125]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
    # Check if the tensor has the expected number of dimensions
        if len(x.shape) != 3:
            raise ValueError("Input tensor must have three dimensions (batch size, sequence length, and feature dimension)")
    
        # Extract the dimensions from the input tensor shape
        batch_size, seq_length, d_model = x.shape
        
        # Reshape the input tensor if necessary
        x = x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
        return x
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).reshape(batch_size, seq_length, -1)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.W_q(Q)
        K = self.W_k(K)
        V = self.W_v(V)
        
        Q = self.split_heads(Q)
        K = self.split_heads(K)
        V = self.split_heads(V)
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))



# Example usage
d_model = 1024
num_heads = 8
d_ff = 2048
dropout = 0.1

encoder_layer = EncoderLayer(d_model, num_heads, d_ff, dropout)

# Sample input and encoder output
x = region_input_tensors  # Input tensor
enc_output = input_full_image.unsqueeze(1).repeat(1, x.size(1), 1)  # Repeat encoder output to match the sequence length

# Pass input through the encoder layer
output = encoder_layer(x, enc_output)

print("Output shape:", output.shape)


RuntimeError: The size of tensor a (1024) must match the size of tensor b (36) at non-singleton dimension 4

In [109]:
import math
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length=512):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerDecoder(nn.Module):
    def __init__(self, input_size, encoder_output_size, d_model=512, nhead=8, num_layers=6, dim_feedforward=2048, dropout=0.1):
        super(TransformerDecoder, self).__init__()
        self.input_size = input_size
        self.encoder_output_size = encoder_output_size
        self.d_model = d_model
        self.nhead = nhead
        self.num_layers = num_layers
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout

        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_seq_length=36)  # Adjust max_seq_length as needed

        self.transformer_layers = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer = nn.TransformerDecoder(self.transformer_layers, num_layers=num_layers)

        self.decoder = nn.Linear(d_model, input_size)

    def forward(self, tgt, encoder_output):
        tgt = self.embedding(tgt) * math.sqrt(self.d_model)
        tgt = self.pos_encoder(tgt)
        output = self.transformer(tgt, encoder_output)
        output = self.decoder(output)
        return output

# Example usage
input_size = 14400  # Adjust to match your input size
encoder_output_size = 1024  # Adjust to match the encoder output size
decoder = TransformerDecoder(input_size, encoder_output_size)

# Generate sample input and encoder output
input_data = region_input_tensors  # Batch size of 4, sequence length of 36
encoder_output = input_full_image  # Encoder output

# Pass input through the decoder
output = decoder(input_data, encoder_output)

print("Output shape:", output.shape)


AssertionError: For batched (3-D) `query`, expected `key` and `value` to be 3-D but found 2-D and 2-D tensors respectively

In [113]:
text = np.random.randint(0, 2048, size=(1, 27))
text.shape

(1, 27)